<a href="https://colab.research.google.com/github/Mohit4Rajput/WRE_.nc/blob/main/Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Restart runtime after running this cell. Use Shift+enter to run individual blocks. 

!pip install importlib-metadata==4.0.1
!pip install xarray==0.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.13.0
    Uninstalling importlib-metadata-4.13.0:
      Successfully uninstalled importlib-metadata-4.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 807 kB 7.5 MB/s 
  Attempting uninstall: xarray
    Found existing installation: xarray 0.20.2
    Uninstalling xarray-0.20.2:
      Successfully uninstalled xarray-0.20.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires xarray>=0.20, but you have xarray 0.18.1 which is incompatible.


In [5]:
# !gdown https://drive.google.com/uc?id=1KInf3dW-jvFUBK75XhsfFlMkaYIgT8MK
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# This cell restarts the runtime
exit()

In [1]:
import xarray as xr
xr.__version__
import netCDF4 as nc

In [2]:
ds = xr.open_dataset('IMD.nc')
ds

FileNotFoundError: ignored

In [ ]:
#IIT Gandhinagar: 27.2046° N, 77.4977° E
iit = ds.sel(lat = 27.2046, lon = 77.4977, method = 'nearest')
iit

<xarray.Dataset>
Dimensions:  (time: 21170)
Coordinates:
    lat      float64 27.25
    lon      float64 77.5
  * time     (time) datetime64[ns] 1948-01-01 1948-01-02 ... 2005-12-31
Data variables:
    rain     (time) float64 ...
Attributes:
    Conventions:  CF-1.7
    title:        IMD gridded data
    source:       https://imdpune.gov.in/
    history:      2022-05-13 23:40:42.719111 Python
    references:   
    comment:      
    crs:          epsg:4326

In [ ]:
ds['pr']

In [ ]:
ds['pr'].plot()

In [ ]:
# Plot daily precipitation data
plt.plot(ds['time'],ds['pr'])
plt.title('Time series of daily observed precipitation')
plt.xlabel("Time", fontsize = 12)
plt.ylabel("Precipitation level (mm/day)", fontsize = 12)

In [ ]:
# plot yearly mean precipitation
observation_yearly = []
for i in range(1950,2006):
    observation_yearly_1 = ds.sel(time=ds.time.dt.year.isin([i])).mean()
    observation_yearly_1 = observation_yearly_1['pr'].values
    observation_yearly.append(observation_yearly_1)
x = np.arange(1950, 2006, dtype=float)
plt.plot(x ,observation_yearly)
plt.title('Time series of yearly observed precipitation')
plt.xlabel("Time", fontsize = 12)
plt.ylabel("Precipitation level (mm/day)", fontsize = 12)

In [ ]:
# plot yearly max precipitation
observation_yearly = []
for i in range(1950,2006):
    observation_yearly_1 = ds.sel(time=ds.time.dt.year.isin([i])).max()
    observation_yearly_1 = observation_yearly_1['pr'].values
    observation_yearly.append(observation_yearly_1)
x = np.arange(1950, 2006, dtype=float)
plt.plot(x ,observation_yearly)
plt.title('Time series of yearly observed precipitation')
plt.xlabel("Time", fontsize = 12)
plt.ylabel("Precipitation level (mm/day)", fontsize = 12)

In [ ]:
from scipy.stats import genextreme
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.genextreme.html

In [ ]:
maxima = ds.groupby('time.year').max('time')
max_list = maxima['pr'].values
print(max_list)

In [ ]:
# Fit the generalized extreme value distribution to the data.
shape, loc, scale = genextreme.fit(max_list)
print(shape)
print(loc)
print(scale)

In [ ]:
# Compute the return levels for return period.
return_periods = 50
return_levels = genextreme.isf(1/return_periods, shape, loc, scale)
print(return_levels)

In [ ]:
def return_level_calculation(max_list,return_periods):
    # Fit the generalized extreme value distribution to the data.
    shape, loc, scale = genextreme.fit(max_list)
    # Compute the return levels for several return periods.
    return_levels = genextreme.isf(1/return_periods, shape, loc, scale)
    return [return_levels]

In [ ]:
# rolling sum of two days
abc = ds.rolling(time=2, center=True).sum()
maxima3 = abc.groupby('time.year').max('time') ['pr'].values
return_levels= return_level_calculation(maxima3, 50)
print(return_levels)

In [ ]:
max_list = []
maxday = [1,2,3,5,10]
return_periods = np.array([ 30, 50, 100])
plt.figure(figsize = (15,15))
plt.rcParams.update({'font.size':20})
for j, period1 in enumerate(return_periods):
    data1 =[]
    for k,daymax1 in enumerate(maxday):
        abc = ds.rolling(time=daymax1, center=True).sum()
        maxima3 = abc.groupby('time.year').max('time') ['pr'].values
        return_levels= return_level_calculation(maxima3, period1)
        #print(return_levels)
        data1.append(return_levels)
    plt.plot(maxday ,data1, label = f"{return_periods[j]}", marker = '*' )
    #max_list.append(maxima3)
plt.legend()
plt.grid()
plt.xlabel("Duration (days)")
plt.ylabel("Precipitation level (mm/day)")
plt.title('DDF curve submitted by (roll number)')
plt.savefig('DDF_curve_roll_number.png')

In [ ]:
# scikit-extremes library
maxima = ds.groupby('time.year').max('time')
max_list = maxima['pr'].values
model = ske.models.classic.GEV(max_list, fit_method = 'mle', ci = 0.05,
                              ci_method = 'delta')

In [ ]:
model.plot_return_values()

In [ ]:
model.plot_return_values()

In [ ]:
model.params
# code for extracting the JJAS - 
# observation = ds.sel(time=ds.time.dt.month.isin([6,7,8,9]))
